In [11]:
# dict comprehension
char_ascii = { chr(i): i for i in range(65, 70)}
char_ascii

{'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69}

In [12]:
# unpacking

{'a': 1 , **{'x':24, 'y': 25}, 'b':2}

{'a': 1, 'x': 24, 'y': 25, 'b': 2}

In [17]:
d1 = { chr(i): i for i in range(65, 70)}
d2 = { chr(i): i for i in range(68, 73)}

# union of dict
d1 | d2

{'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72}

In [ ]:
# inplace union
d1 |= d2
d1

{'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72}

In [ ]:
# Pattern matching

def get_book(record: dict) -> list:
    match record:
        case {'book': 'maths', 'api': 1, 'authors': name}: 
            print("maths", end=" ")
            return [name]
        case {'book': 'science', 'api': 1, 'authors': [*names]}:
            print("science", end=" ")
            return names
        case {'book': 'physics', 'api': 2, 'authors': str(name)}:
            print("physics", end=" ")
            return [name]
        case {'book': 'chemistry', 'api': 2, 'authors': list(names)}:
            print("chemistry", end=" ")
            return names
        case {'book': 'computer', **details}:
            print("computer", end=" ")
            return details.get('authors', [])
            
        case _: 
            raise ValueError(f"Invalid book {record!r}")
    

print(get_book({'book': 'maths', 'api': 1, 'authors': 'Coco'}))
print(get_book({'book': 'physics', 'api': 2, 'authors': 'Dora'}))
print(get_book({'book': 'chemistry', 'api': 2, 'authors': ['Giner', 'Niko', 'possom bro']}))
print(get_book({'book': 'computer', 'api': 1}))


maths ['Coco']
physics ['Dora']
chemistry ['Giner', 'Niko', 'possom bro']
computer []


In [36]:
import collections
import collections.abc as abc
isinstance(d1, abc.Mapping)

True

In [35]:
isinstance(d1, abc.MutableMapping)

True

In [49]:
from collections import UserDict

class CaseInsensitiveDict(UserDict):
    def __init__(self, *args, **kwargs):
        self._keymap = {}  # maps lowercase key → original key
        super().__init__(*args, **kwargs)

    def _lower(self, key):
        return key.lower() if isinstance(key, str) else key

    def __setitem__(self, key, value):
        lower_key = self._lower(key)
        if lower_key in self._keymap:
            actual_key = self._keymap[lower_key]
        else:
            actual_key = key
            self._keymap[lower_key] = key
        self.data[actual_key] = value

    def __getitem__(self, key):
        lower_key = self._lower(key)
        actual_key = self._keymap[lower_key]
        return self.data[actual_key]

    def __delitem__(self, key):
        lower_key = self._lower(key)
        actual_key = self._keymap.pop(lower_key)
        del self.data[actual_key]

    def __contains__(self, key):
        return self._lower(key) in self._keymap

    def get(self, key, default=None):
        return self[key] if key in self else default


In [51]:
cid = CaseInsensitiveDict()

cid["Host"] = "localhost"
cid["User-Agent"] = "Mozilla"
cid["Content-Type"] = "application/json"

print(cid["host"])         # localhost
print(cid["USER-AGENT"])   # Mozilla

print("HOST" in cid)       # True

cid["host"] = "127.0.0.1"  # overwrite

print(cid)  # original keys preserved!


localhost
Mozilla
True
{'Host': '127.0.0.1', 'User-Agent': 'Mozilla', 'Content-Type': 'application/json'}


### Hashable

In [53]:
t = (1,2, 3)
hash(t)

529344067295497451

In [ ]:
print(hash(10))           
print(hash("hello"))      
print(hash((1, 2, 3)))
print(hash([1, 2, 3]))     # TypeError: unhashable type: 'list'


10
-8048573879111871024
529344067295497451


TypeError: unhashable type: 'list'

In [57]:
t = (1,2, [3, 4])
hash(t)

TypeError: unhashable type: 'list'

In [59]:
# Check If Something Is Hashable
from collections.abc import Hashable

print(isinstance(10, Hashable))        # True
print(isinstance([1, 2], Hashable))    # False
print(isinstance((1, 2), Hashable))    # True


True
False
True


In [ ]:
# dict.setDefault

# setdefault(key, default) — Lookup AND set

#     Returns the value if key exists

#     Else inserts key = default into the dictionary

#     Modifies the dictionary if the key is missing

words = ['apple', 'banana', 'apricot', 'blueberry']

grouped = {}

for word in words:
    first_letter = word[0]
    grouped.setdefault(first_letter, []).append(word)

print(grouped)


{'a': ['apple', 'apricot'], 'b': ['banana', 'blueberry']}


In [62]:
import dis
dis.dis("grouped.setdefault(first_letter, []).append(word)")

  1           0 LOAD_NAME                0 (grouped)
              2 LOAD_METHOD              1 (setdefault)
              4 LOAD_NAME                2 (first_letter)
              6 BUILD_LIST               0
              8 CALL_METHOD              2
             10 LOAD_METHOD              3 (append)
             12 LOAD_NAME                4 (word)
             14 CALL_METHOD              1
             16 RETURN_VALUE


In [74]:
class strKeytDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self or str(key) in self
    

d = strKeytDict0({'2': 'two', '4': 'four'})

In [67]:
d['2']

'two'

In [66]:
d[4]

'four'

In [68]:
d['5']

KeyError: 

In [69]:
# defaultdict

dd = collections.defaultdict(int)

for i in range(2, 11):
    if i%2 == 0:
        dd['even'] += 1
    else:
        dd['odd'] += 1

dd  

defaultdict(int, {'even': 5, 'odd': 4})

In [ ]:
# collections.chainmap 
#  roups multiple dicts or other mappings together to create a single, updateable view


d1 = dict(a=1, b=1)
d2 = dict(c=3, d=4, e=5)
chain_dict = collections.ChainMap(d1, d2)

chain_dict['d']

4

In [85]:
# Subclassing collections.UserDict

class StrkeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError
        return self[str(key)]
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item
    
    def __contains__(self, key):
        print(self)
        return str(key) in self.data
    
    def is_self_data(self):
        print(self is self.data)
        print(self)
        print(self.data)

In [86]:
sd = StrkeyDict({'2': 'two', '4': 'four'})
sd['3']= 'three'
sd 

{'2': 'two', '4': 'four', '3': 'three'}

In [78]:
sd.get('3')

'three'

In [87]:
sd.is_self_data()

False
{'2': 'two', '4': 'four', '3': 'three'}
{'2': 'two', '4': 'four', '3': 'three'}


In [89]:
# Dictionary views

d = dict(a=10, b=20, c=30)
keys = d.keys()
keys

dict_keys(['a', 'b', 'c'])

In [90]:
values = d.values()
values

dict_values([10, 20, 30])

In [91]:
values[0]

TypeError: 'dict_values' object is not subscriptable

In [94]:
d['z'] = 99
d

{'a': 10, 'b': 20, 'c': 30, 'z': 99}

In [96]:
# A view object is dynamic proxy
values

dict_values([10, 20, 30, 99])

### Sets

In [99]:
l = [1, 2, 3, 2, 1, 4 ,5 , 6, 5]
set(l)

{1, 2, 3, 4, 5, 6}

In [104]:
# set literal
s = {1}

type(s)

set

In [105]:
print(s.pop())
s

1


set()

In [108]:
# set comphrensions

import unicodedata

unicodedata.name('a')

'LATIN SMALL LETTER A'

In [119]:
{chr(i) for i in range(32, 256) if 'SIGN' in unicodedata.name(chr(i), "")}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [118]:
unicodedata.name('b')

'LATIN SMALL LETTER B'